#giving the csv containing Entry, CATH number, and Sequence, retrive the esm1b embeddings

In [3]:
from CLEAN import *

ModuleNotFoundError: No module named 'CLEAN'

In [ ]:
from CLEAN import *
ensure_dirs("data/esm_data")

ensure_dirs("data/pretrained")

csv_to_fasta("data/extra_cath_training_data_pisces.csv", "data/extra_cath_training_data_pisces.fasta")

retrive_esm1b_embedding("extra_cath_training_data_pisces")

As the esm1b can only take sequence shorter than 1024, the large sequences are chunked by size of 800 aa. This step will group the files that are chunked, concatenate the tensor without changing dimension, and average them. 

In [ ]:
import os
import torch
from collections import defaultdict
import re

dir_path = "/Users/Zachary/Desktop/Dissertation/CLEAN/app/data/esm_data"
output_path = "/Users/Zachary/Desktop/Dissertation/CLEAN/app/data/concatenated_chunked_esm_data"

all_files = [f for f in os.listdir(dir_path) if f.endswith(".pt")]

file_groups = defaultdict(list)
for file_name in all_files:
    root_name = file_name.split('_')[0]
    file_groups[root_name].append(file_name)

for root, files in file_groups.items():
    if len(files) > 1:
        files.sort(key=lambda f: int(f.split('_')[-1].split('.')[0]))

        sum_tensor = None
        original_dict = None

        for file in files:
            file_path = os.path.join(dir_path, file)
            loaded_dict = torch.load(file_path)
            tensor = loaded_dict['mean_representations'][33]

            if sum_tensor is None:
                sum_tensor = tensor
                original_dict = loaded_dict
            else:
                sum_tensor += tensor

        average_tensor = sum_tensor / len(files)
        original_dict['mean_representations'][33] = average_tensor

        output_file_name = f"{root}.pt"
        output_file_path = os.path.join(output_path, output_file_name)
        torch.save(original_dict, output_file_path)
        print(f"Merged {output_file_name}")

    else:
        file_name = files[0]
        input_file_path = os.path.join(dir_path, file_name)
        output_file_path = os.path.join(output_path, file_name)
        torch.save(torch.load(input_file_path), output_file_path)
        print(f"Copied {file_name}")

# Iterate through all the .pt files in the directory and keep the file name in length of 4
for file_name in os.listdir(output_path):
    if file_name.endswith(".pt"):
        file_path = os.path.join(output_path, file_name)
        
        # Load the .pt file
        loaded_dict = torch.load(file_path)
        
        # Modify the 'label' key value
        loaded_dict['label'] = loaded_dict['label'][:4]
        
        # Save the updated dictionary to the same file
        torch.save(loaded_dict, file_path)
        print(f"Updated label for {file_name}")

A test code to verify the content in the new pt file is correct. 

In [ ]:
sample_file_1 = torch.load("/Users/Zachary/Desktop/Dissertation/CLEAN/app/data/esm_data/5zi1_1.pt")
sample_file_2 = torch.load("/Users/Zachary/Desktop/Dissertation/CLEAN/app/data/esm_data/5zi1_2.pt")
sample_file_3 = torch.load("/Users/Zachary/Desktop/Dissertation/CLEAN/app/data/concatenated_chunked_esm_data/5zi1.pt")
print (sample_file_1,sample_file_2,sample_file_3,sample_file_2['mean_representations'][33].shape)

Before training, a required step is to mutate the sequences with 'orphan' EC number ('orphan' in the way this EC number has only one sequence). Since we need to sample positive sequences other than the anchor sequence, we mutated the anchor sequence and use the mutated sequences as positive sequences. 

In [ ]:
from CLEAN.utils import mutate_single_seq_ECs, retrive_esm1b_embedding

train_file = "extra_cath_training_data_pisces"

train_fasta_file = mutate_single_seq_ECs(train_file)

retrive_esm1b_embedding(train_fasta_file)

#from CLEAN import *
#csv_to_fasta("data/extra_cath_training_data_pisces.csv", "data/extra_cath_training_data_pisces.fasta")
#
#retrive_esm1b_embedding("extra_cath_training_data_pisces")

After retriving the embeddings, the triplicated rows with sequences longer than 1024 have been chunked.

Therefore, we need to concatenate and average the tensor again

In [ ]:
import os
import torch
from collections import defaultdict
import re

dir_path = "/Users/Zachary/Desktop/Dissertation/CLEAN/app/data/esm_data"
output_path = "/Users/Zachary/Desktop/Dissertation/CLEAN/app/data/concatenated_chunked_triplicated_esm_data"

all_files = [f for f in os.listdir(dir_path) if f.endswith(".pt")]
file_groups = defaultdict(list)

for file_name in all_files:
    split_name = file_name.split('_')
    
    if len(split_name) >= 2:
        root_name = split_name[0] + '_' + split_name[1].split('.')[0]
    else:
        root_name = split_name[0].split('.')[0]

    file_groups[root_name].append(file_name)

for root, files in file_groups.items():
    if len(files) > 1:
        files.sort(key=lambda f: int(f.split('_')[-1].split('.')[0]))

        sum_tensor = None
        original_dict = None

        for file in files:
            file_path = os.path.join(dir_path, file)
            loaded_dict = torch.load(file_path)
            tensor = loaded_dict['mean_representations'][33]

            if sum_tensor is None:
                sum_tensor = tensor
                original_dict = loaded_dict
            else:
                sum_tensor += tensor

        average_tensor = sum_tensor / len(files)
        original_dict['mean_representations'][33] = average_tensor

        output_file_name = f"{root}.pt"
        output_file_path = os.path.join(output_path, output_file_name)
        torch.save(original_dict, output_file_path)
        print(f"Merged {output_file_name}")

    else:
        file_name = files[0]
        input_file_path = os.path.join(dir_path, file_name)
        output_file_path = os.path.join(output_path, file_name)
        torch.save(torch.load(input_file_path), output_file_path)
        print(f"Copied {file_name}")

# Iterate through all the .pt files in the directory and keep the file name in length of 4
for file_name in os.listdir(output_path):
    if file_name.endswith(".pt"):
        file_path = os.path.join(output_path, file_name)
        
        # Load the .pt file
        loaded_dict = torch.load(file_path)
        #modify
        if len(loaded_dict['label']) > 6:
            loaded_dict['label'] = loaded_dict['label'][:6]
        # Save the updated dictionary to the same file
        torch.save(loaded_dict, file_path)
        print(f"Updated label for {file_name}")

change the name of concatenated_chunked_triplicated_esm_data to esm_data 


To speed up training, the pair-wise distance matrix and embedding matrix need to be pre-computed

In [ ]:
from CLEAN.utils import compute_esm_distance

train_file = "short_cath_training_data_pisces"

compute_esm_distance(train_file)


Train the model with triplet margin loss

In [1]:
!python ./train-triplet.py --training_data short_cath_training_data_pisces --model_name short_cath_training_data_pisces --epoch 6500

^C
Fatal Python error: init_import_site: Failed to import the site module
Python runtime state: initialized
Traceback (most recent call last):
  File "/Users/Zachary/opt/anaconda3/lib/python3.9/site.py", line 589, in <module>
    main()
  File "/Users/Zachary/opt/anaconda3/lib/python3.9/site.py", line 576, in main
    known_paths = addsitepackages(known_paths)
  File "/Users/Zachary/opt/anaconda3/lib/python3.9/site.py", line 359, in addsitepackages
    addsitedir(sitedir, known_paths)
  File "/Users/Zachary/opt/anaconda3/lib/python3.9/site.py", line 203, in addsitedir
    names = os.listdir(sitedir)
KeyboardInterrupt


Train the model with SupCon-Hard loss

In [1]:

!python ./train-supconH.py --training_data short_cath_training_data_pisces --model_name cath_training_data_pisces_supconH --epoch 4875 --n_pos 9 --n_neg 30 -T 0.1

==> device used: cpu | dtype used:  torch.float32 
==> args: Namespace(learning_rate=0.0005, epoch=4875, model_name='cath_training_data_pisces_supconH', training_data='short_cath_training_data_pisces', temp=0.1, n_pos=9, n_neg=30, hidden_dim=512, out_dim=256, adaptive_rate=60, verbose=False)
The number of unique EC numbers:  1511
---------------------------------------------------------------------------
| end of epoch   1 | time:  9.72s | training loss 7.4506
---------------------------------------------------------------------------
---------------------------------------------------------------------------
| end of epoch   2 | time:  7.05s | training loss 5.4803
---------------------------------------------------------------------------
---------------------------------------------------------------------------
| end of epoch   3 | time:  7.18s | training loss 5.5312
---------------------------------------------------------------------------
-----------------------------------------

In [ ]:
from CLEAN.utils import *
#new is the the name of the file that you want to get inference on

csv_to_fasta("data/same_test_1.csv", "data/same_test_1.fasta")

retrive_esm1b_embedding("same_test_1")


from CLEAN.infer import infer_pvalue

train_data = "short_cath_training_data_pisces"

test_data = "same_test_1"

infer_pvalue(train_data, test_data, p_value=1e-5, nk_random=20, report_metrics=True, pretrained=False,model_name="short_cath_training_data_pisces")


from CLEAN.infer import *

infer_maxsep("short_cath_training_data_pisces", "same_test_1", report_metrics=True, pretrained=False, model_name="short_cath_training_data_pisces")


In [ ]:
csv_to_fasta("data/same_test_2.csv", "data/same_test_2.fasta")

retrive_esm1b_embedding("same_test_2")


from CLEAN.infer import infer_pvalue

train_data = "short_cath_training_data_pisces"

test_data = "same_test_2"

infer_pvalue(train_data, test_data, p_value=1e-5, nk_random=20,

report_metrics=True, pretrained=False,model_name="short_cath_training_data_pisces")


from CLEAN.infer import *

infer_maxsep("short_cath_training_data_pisces", "same_test_2", report_metrics=True,

pretrained=False, model_name="short_cath_training_data_pisces")

In [ ]:
csv_to_fasta("data/same_test_3.csv", "data/same_test_3.fasta")

retrive_esm1b_embedding("same_test_3")

from CLEAN.infer import infer_pvalue

train_data = "short_cath_training_data_pisces"

test_data = "same_test_3"

infer_pvalue(train_data, test_data, p_value=1e-5, nk_random=20,

report_metrics=True, pretrained=False,model_name="short_cath_training_data_pisces")


from CLEAN.infer import *

infer_maxsep("short_cath_training_data_pisces", "same_test_3", report_metrics=True,

pretrained=False, model_name="short_cath_training_data_pisces")

In [ ]:

from CLEAN.infer import *

infer_maxsep("short_cath_training_data_pisces", "same_test_3", report_metrics=True,

pretrained=False, model_name="short_cath_training_data_pisces")

In [ ]:
csv_to_fasta("data/same_test_4.csv", "data/same_test_4.fasta")

retrive_esm1b_embedding("same_test_4")


from CLEAN.infer import infer_pvalue

train_data = "short_cath_training_data_pisces"

test_data = "same_test_4"

infer_pvalue(train_data, test_data, p_value=1e-5, nk_random=20,

report_metrics=True, pretrained=False,model_name="short_cath_training_data_pisces")


from CLEAN.infer import *

infer_maxsep("short_cath_training_data_pisces", "same_test_4", report_metrics=True,

pretrained=False, model_name="short_cath_training_data_pisces")

In [ ]:
from CLEAN.infer import *

infer_maxsep("short_cath_training_data_pisces", "same_test_4", report_metrics=True,

pretrained=False, model_name="short_cath_training_data_pisces")

see the result

In [ ]:
import pandas as pd
def compare_columns(file1, file2, col1, col2, num_chars):
    df1 = pd.read_csv('/Users/Zachary/Desktop/Dissertation/CLEAN/app/results/' + file1 + '.csv')
    df2 = pd.read_csv('/Users/Zachary/Desktop/Dissertation/CLEAN/app/data/' + file2 + '.csv',sep='\t')
    df1[col1] = df1[col1].str[3:]

    # Extract the values from the columns based on the input digit
    df1[col1 + ' first ' + str(num_chars)] = df1[col1].str.split('.').str[:num_chars].str.join('.')
    df2[col2 + ' first ' + str(num_chars)] = df2[col2].str.split('.').str[:num_chars].str.join('.')

    # Compare the values and create a new 'comparison' column
    df1['comparison'] = df1[col1 + ' first ' + str(num_chars)] == df2[col2 + ' first ' + str(num_chars)]

    # Filter the DataFrame based on the 'comparison' column
    true_entries = df1[df1['comparison'] == True]
    false_entries = df1[df1['comparison'] == False]

    # Extract the 'Entry' values as lists
    true_entry_list = true_entries['Entry'].tolist()
    false_entry_list = false_entries['Entry'].tolist()

    # Get the number of elements in each list
    true_entry_count = len(true_entry_list)
    false_entry_count = len(false_entry_list)
    
    print("Number of true entries:", true_entry_count)
    #print ("True entries:", true_entry_list)
    print("Number of false entries:", false_entry_count)


In [ ]:
compare_columns('same_test_1_maxsep', 'same_test_1', 'CATH number 2', 'EC number', 1)

In [ ]:
compare_columns('same_test_2_maxsep', 'same_test_2', 'CATH number 4', 'EC number', 1)

In [ ]:
compare_columns('same_test_3_maxsep', 'same_test_3', 'CATH number 1', 'EC number', 1)

In [ ]:
compare_columns('same_test_4_maxsep', 'same_test_4', 'CATH number 2', 'EC number', 2)

In [ ]:
same_test_2_maxsep = pd.read_csv('/Users/Zachary/Desktop/Dissertation/CLEAN/app/results/same_test_2_maxsep.csv', header=0)
sample_test_2= pd.read_csv('/Users/Zachary/Desktop/Dissertation/CLEAN/app/data/same_test_2.csv',sep='\t')

same_test_2_maxsep['CATH number 1'] = same_test_2_maxsep['CATH number 1'].str[3:]

same_test_2_maxsep['comparison'] = same_test_2_maxsep['CATH number 1'] == sample_test_2['EC number']
same_test_2_maxsep

# Filter the DataFrame based on the 'comparison' column
true_entries = same_test_2_maxsep[same_test_2_maxsep['comparison'] == True]
false_entries = same_test_2_maxsep[same_test_2_maxsep['comparison'] == False]

# Extract the 'Entry' values as lists
true_entry_list = true_entries['Entry'].tolist()
false_entry_list = false_entries['Entry'].tolist()

# Get the number of elements in each list
true_entry_count = len(true_entry_list)
false_entry_count = len(false_entry_list)

# Print the results
#print("True entry list:", true_entry_list)
print("Number of true entries:", true_entry_count)

#print("False entry list:", false_entry_list)
print("Number of false entries:",false_entry_count)


In [ ]:
# Extract the first three values from the 'CATH number 1' and 'EC number' columns
same_test_2_maxsep['CATH number 1 first three'] = same_test_2_maxsep['CATH number 1'].str.split('.').str[:3].str.join('.')
sample_test_2['EC number first three'] = sample_test_2['EC number'].str.split('.').str[:3].str.join('.')

# Compare the first three values and create a new 'comparison' column
same_test_2_maxsep['comparison'] = same_test_2_maxsep['CATH number 1 first three'] == sample_test_2['EC number first three']

# Filter the DataFrame based on the 'comparison' column
true_entries = same_test_2_maxsep[same_test_2_maxsep['comparison'] == True]
false_entries = same_test_2_maxsep[same_test_2_maxsep['comparison'] == False]

# Extract the 'Entry' values as lists
true_entry_list = true_entries['Entry'].tolist()
false_entry_list = false_entries['Entry'].tolist()

# Get the number of elements in each list
true_entry_count = len(true_entry_list)
false_entry_count = len(false_entry_list)

# Print the results
#print("True entry list:", true_entry_list)
print("Number of true entries:", true_entry_count)

#print("False entry list:", false_entry_list)
print("Number of false entries:", false_entry_count)


In [ ]:
# Extract the first two values from the 'CATH number 1' and 'EC number' columns
same_test_2_maxsep['CATH number 1 first three'] = same_test_2_maxsep['CATH number 1'].str.split('.').str[:2].str.join('.')
sample_test_2['EC number first three'] = sample_test_2['EC number'].str.split('.').str[:2].str.join('.')

# Compare the first three values and create a new 'comparison' column
same_test_2_maxsep['comparison'] = same_test_2_maxsep['CATH number 1 first three'] == sample_test_2['EC number first three']

# Filter the DataFrame based on the 'comparison' column
true_entries = same_test_2_maxsep[same_test_2_maxsep['comparison'] == True]
false_entries = same_test_2_maxsep[same_test_2_maxsep['comparison'] == False]

# Extract the 'Entry' values as lists
true_entry_list = true_entries['Entry'].tolist()
false_entry_list = false_entries['Entry'].tolist()

# Get the number of elements in each list
true_entry_count = len(true_entry_list)
false_entry_count = len(false_entry_list)

# Print the results
#print("True entry list:", true_entry_list)
print("Number of true entries:", true_entry_count)

#print("False entry list:", false_entry_list)
print("Number of false entries:", false_entry_count)


Inference with p-value

Inference with max-separation

In [ ]:
from CLEAN.infer import *

infer_maxsep("short_cath_training_data_pisces", "same_test_1", report_metrics=True,

pretrained=False, model_name="short_cath_training_data_pisces")

In [ ]:
import torch
import os


directory = '/Users/Zachary/Desktop/Dissertation/CLEAN/app/data/esm_data'

# Iterate through all files in the directory
for filename in os.listdir(directory):
    # Check if the file is a .pt file
    if filename.endswith('.pt'):
        filepath = os.path.join(directory, filename)
        # Load the .pt file using PyTorch
        data = torch.load(filepath)

        # Extract the tensor
        mean_representations = data['mean_representations']

        # Check for zero values in each tensor
        zero_values_present = False
        for key, tensor in mean_representations.items():
            if torch.any(tensor == 0):
                zero_values_present = True
                print(f"File {filename} contains zero values in tensor with key {key}.")
                break

        

In [ ]:
import torch
import pickle


In [ ]:
# Path to your .pkl file
pkl_file_path = '/Users/Zachary/Desktop/Dissertation/CLEAN/app/data/distance_map/short_cath_training_data_pisces.pkl'

# Open the file in binary read mode
with open(pkl_file_path, 'rb') as file:
    # Load the contents of the file using pickle
    data = pickle.load(file)

# Now you can access the data in the pickle file
print(data)


In [ ]:
tensor = data

# Check if there's any zero value in the tensor
contains_zero = torch.any(tensor == 0)

# Print the result
if contains_zero:
    print("The tensor contains at least one zero value.")
else:
    print("The tensor does not contain any zero values.")

In [ ]:
content = torch.load("/Users/Zachary/Desktop/Dissertation/CLEAN/app/data/model/cath_training_data_pisces_triplet_6500.pth")
content

result of dataset 3

In [ ]:
# Read CSV files
sample3_result = pd.read_csv('/Users/Zachary/Desktop/Dissertation/CLEAN/app/results/same_test_3_maxsep.csv')
wholedata_label = pd.read_csv('./core_database/destress_wholedata_labelled_as_rd.csv')

sample3_result1=sample3_result.copy()
sample3_result1['CATH number 3'] = sample3_result1['CATH number 3'].str[3:]
sample_labelling = wholedata_label.merge(sample3_result1, on='Entry', how='left')
sample_labelling
test_500 = sample_labelling[['Entry','label_3','CATH number 3']]
test_500['label_4'] = test_500['label_3'].str.split('.').str[:4].str.join('.')
test_500['label_5'] = test_500['CATH number 3'].str.split('.').str[:4].str.join('.')
test_500.dropna(subset=['CATH number 3'], inplace=True)
test_500
test_500['Comparison'] = test_500['label_4'] == test_500['label_5']

true_rows = test_500[test_500['Comparison'] == True]
true_rows.shape